In [1]:
!pip install tarfile


ERROR: Could not find a version that satisfies the requirement tarfile (from versions: none)
ERROR: No matching distribution found for tarfile


In [14]:
HAM = 0
SPAM = 1
datadir = 'data'
sources = [
    ('beck-s.tar.gz', HAM),
    ('farmer-d.tar.gz', HAM),
    ('kaminski-v.tar.gz', HAM),
    ('kitchen-l.tar.gz',HAM),
    ('lokay-m.tar.gz',HAM),
    ('williams-w3.tar.gz',HAM),
    ('BG.tar.gz', SPAM),
    ('GP.tar.gz', SPAM),
    ('SH.tar.gz', SPAM)
]

In [6]:
def extract_tar(datafile, extractdir):
    try:
        import tarfile
    except ImportError:
        raise ImportError("You don't have tarfie installed. "
                          "Try unzipping the file outside of python.")

    tar = tarfile.open(datafile)
    tar.extractall(path=extractdir)
    tar.close()
    print("%s Successfully extracted to %s" % (datafile, extractdir))

In [15]:
for source, _ in sources:
    datafile = '%s/%s' % (datadir, source)
    extract_tar(datafile, datadir)

data/beck-s.tar.gz Successfully extracted to data
data/farmer-d.tar.gz Successfully extracted to data
data/kaminski-v.tar.gz Successfully extracted to data
data/kitchen-l.tar.gz Successfully extracted to data
data/lokay-m.tar.gz Successfully extracted to data
data/williams-w3.tar.gz Successfully extracted to data
data/BG.tar.gz Successfully extracted to data
data/GP.tar.gz Successfully extracted to data
data/SH.tar.gz Successfully extracted to data


In [18]:
import os
def read_single_file(filename):
    past_header, lines = False, []
    if os.path.isfile(filename):
        f = open(filename, encoding="latin-1")
        for line in f:
            if past_header:
                lines.append(line)
            elif line == '\n':
                past_header = True
        f.close()
    content = '\n'.join(lines)
    return filename, content

In [20]:
def read_files(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            filepath = os.path.join(root, filename)
            yield read_single_file(filepath)

In [22]:
import pandas as pd
pd. DataFrame({
    'model': ['Normal Bayes', 'Multinomial Bayes', 'Bernoulli Bayes'],
    'class': [
        'cv2.ml.NormalBayesClassifier_create()',
        'sklearn.naive_bayes.MultinomialNB()',
        'sklearn.naive_bayes.BernolliNB()' ]
})

,model,class
0,Normal Bayes,cv2.ml.NormalBayesClassifier_create()
1,Multinomial Bayes,sklearn.naive_bayes.MultinomialNB()
2,Bernoulli Bayes,sklearn.naive_bayes.BernolliNB()


In [23]:
def build_data_frame(extractdir, classification):
    rows = []
    index = []
    for file_name, text in read_files(extractdir):
        rows.append({'text': text, 'class': classification})
        index.append(file_name)

    data_frame =pd.DataFrame(rows, index=index)
    return data_frame

In [24]:
data = pd.DataFrame({'text': [], 'class': []})
for source, classification in sources:
    extractdir = '%s/%s' % (datadir, source[:-7])
    data = data.append(build_data_frame(extractdir, classification))

<ipython-input-24-7bc3a618ec6a>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(build_data_frame(extractdir, classification))
<ipython-input-24-7bc3a618ec6a>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(build_data_frame(extractdir, classification))
<ipython-input-24-7bc3a618ec6a>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(build_data_frame(extractdir, classification))
<ipython-input-24-7bc3a618ec6a>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(build_data_frame(extractdir, classification))
<ipython-input-24-7bc3a618ec6a>:4: FutureWarning: The frame.

In [25]:
from sklearn import feature_extraction
counts = feature_extraction.text.CountVectorizer()
X = counts.fit_transform(data['text'].values)
X.shape

(52076, 643270)

In [26]:
X

<52076x643270 sparse matrix of type '<class 'numpy.int64'>'
	with 8607632 stored elements in Compressed Sparse Row format>

In [29]:
y = data['class'].values
y

array([0., 0., 0., ..., 1., 1., 1.])

In [30]:
from sklearn import model_selection as ms 
X_train, X_test, y_train, y_test = ms.train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [31]:
import cv2 
model_norm = cv2.ml.NormalBayesClassifier_create()

In [32]:
import numpy as np
X_train_small = X_train[:1000, :300].toarray().astype(np.float32)
y_train_small = y_train[:1000]

In [33]:
X_train, X_test, y_train, y_test = ms.train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [34]:
from sklearn import naive_bayes
model_naive = naive_bayes.MultinomialNB()
model_naive.fit(X_train, y_train)

MultinomialNB()

In [45]:
model_naive.score(X_train, y_train)

0.9960873739798368

In [36]:
model_naive.score(X_test,y_test)

0.9449884792626728

In [37]:
counts = feature_extraction.text.CountVectorizer(
    ngram_range=(1, 2)
)
X = counts.fit_transform(data['text'].values)

In [38]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [39]:
model_naive = naive_bayes.MultinomialNB()
model_naive.fit(X_train,y_train)

MultinomialNB()

In [40]:
model_naive.score(X_test, y_test)

0.9718701996927803

In [41]:
tfidf = feature_extraction.text.TfidfTransformer()
X_new = tfidf.fit_transform(X)

In [42]:
X_train, X_test, y_train, y_test = ms.train_test_split(
    X_new, y, test_size=0.2, random_state=42
)

In [43]:
model_naive = naive_bayes.MultinomialNB()
model_naive.fit(X_train, y_train)
model_naive.score(X_test, y_test)

0.9910714285714286

In [44]:
from sklearn import metrics
metrics.confusion_matrix(y_test, model_naive.predict(X_test))

array([[3745,   85],
       [   8, 6578]])